# Plotting Means and Error Ranges

## Error Ranges

There are several ways to show error ranges on a plot:
- `geom_errorbar()`
- `geom_crossbar()`
- `geom_linerange()`
- `geom_pointrange()`

In [1]:
from lets_plot import *

In [2]:
LetsPlot.setup_html()

### The "Tooth Growth" Dataset

The ToothGrowth dataset describes the effect of Vitamin C on tooth growth in guinea pigs. Each animal received one of three dose levels of vitamin C (0.5, 1, and 2 mg/day) by one of two delivery methods: orange juice (OJ) or ascorbic acid (VC).

* len : Tooth length
* dose : Dose in milligrams (0.5, 1, 2)
* supp : Supplement type (VC or OJ)

Let's calculate the mean value of tooth length in each group, minimum and maximum values, and use this information to plot error bars.

In [3]:
def get_data():
    import pandas as pd
    raw_df = pd.read_csv('https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/ToothGrowth.csv')
    result = raw_df.groupby(by=['supp', 'dose']).agg({'len': ['min', 'max', 'median']}).reset_index()
    result.columns = ['supp', 'dose', 'len_min', 'len_max', 'length']
    return result

In [4]:
df = get_data()
df

,supp,dose,len_min,len_max,length
0,OJ,0.5,8.2,21.5,12.25
1,OJ,1.0,14.5,27.3,23.45
2,OJ,2.0,22.4,30.9,25.95
3,VC,0.5,4.2,11.5,7.15
4,VC,1.0,13.6,22.5,16.50
5,VC,2.0,18.5,33.9,25.95


In [5]:
p1 = ggplot(df, aes(x='dose', color='supp'))

### Error Bars with Lines and Points

`geom_errorbar()` displays error bars defined by the upper and lower values.

In [6]:
p1 + geom_errorbar(aes(ymin='len_min', ymax='len_max'), width=.1) + \
    geom_line(aes(y='length')) + \
    geom_point(aes(y='length'))

The error bars overlap, so you can use `position_dodge` to shift them horizontally.

In [7]:
dodge = position_dodge(0.1) # move them .05 to the left and right
p1 + geom_errorbar(aes(ymin='len_min', ymax='len_max'), width=.1, position=dodge) + \
    geom_line(aes(y='length'), position=dodge) + \
    geom_point(aes(y='length'), position=dodge)

The black error bars — notice how the mapping `group='supp'` is applied. Without it, the error bars won't be dodged!

In [8]:
p1 + geom_errorbar(aes(ymin='len_min', ymax='len_max', group='supp'), 
                  color='black', width=.1, position=dodge) + \
    geom_line(aes(y='length'), position=dodge) + \
    geom_point(aes(y='length'), position=dodge, size=5)

The final graph includes:
 - fixed size
 - point shape 21 (filled circle) 
 - position legend in the bottom right

In [9]:
p2 = p1 + \
    xlab('Dose (mg)') + \
    ylab('Tooth length (mm)') + \
    scale_color_manual(['orange', 'dark_green'], na_value='gray') + \
    ggsize(700, 400)

p2 + \
    geom_errorbar(aes(ymin='len_min', ymax='len_max', group='supp'), 
                  color='black', width=.1, position=dodge) + \
    geom_line(aes(y='length'), position=dodge) + \
    geom_point(aes(y='length'), position=dodge, size=5, shape=21, fill='white') + \
    theme(legend_justification=[1,0], legend_position=[1,0]) + \
    ggtitle('The Effect of Vitamin C on Tooth Growth in Guinea Pigs')

### Error Bars on Bar Plot

In [10]:
p2 + \
    geom_bar(aes(y='length', fill='supp'), stat='identity', 
             position='dodge', color='black') + \
    geom_errorbar(aes(ymin='len_min', ymax='len_max', group='supp'), 
                  color='black', width=.1, position=position_dodge(0.9)) + \
    theme(legend_justification=[0,1], legend_position=[0,1])

### Crossbars

`geom_crossbar()` displays bars with a horizontal median line. The thickness of the horizontal mid-line can be adjusted using `fatten` parameter.

In [11]:
p2 + geom_crossbar(aes(ymin='len_min', ymax='len_max', y='length', color='supp'), fatten=5)

### Line Range
`geom_linerange()` displays a line range defined by an upper and lower values.

In [12]:
p2 + \
    geom_linerange(aes(ymin='len_min', ymax='len_max', color='supp'), position=dodge) + \
    geom_line(aes(y='length'), position=dodge)

### Point Range
A point range is like a line range, but with an added mid-point.

In [13]:
p2 + \
    geom_pointrange(aes(y='length', ymin='len_min', ymax='len_max', color='supp'), position=dodge) + \
    geom_line(aes(y='length'), position=dodge)

The size of the mid-point can be adjusted using `fatten` parameter, a multiplication factor relative to the line size.

In [14]:
p2 + \
    geom_line(aes(y='length'), position=dodge) + \
    geom_pointrange(aes(y='length', ymin='len_min', ymax='len_max', fill='supp'), 
                    position=dodge, color='rgb(128, 128, 128)', size=6, shape=23, fatten=1) + \
    scale_fill_manual(['orange', 'dark_green'], na_value='gray')

## Geometries with Dual Orientation

- `geom_linerange()`
- `geom_pointrange()`
- `geom_errorbar()`
- `geom_crossbar()`
- `geom_ribbon()`

You can flip orientation of these geometries simply by flipping their positional aesthetics mapping.

### Flipping an Error Bar

`geom_errorbar()` can be plotted horizontally by assigning `y`, `xmin`, `xmax` aesthetics. The height of the error bar is defined by the `height`.

New type of position adjustment `'dodgev'` is used to adjust the position by dodging overlaps to the side. Function `position_dodgev(height)` allows to set the dodge height.

In [15]:
gggrid([
    ggplot(df, aes(color='supp')) + \
        geom_errorbar(aes(x='dose', ymin='len_min', ymax='len_max'), 
                      position=position_dodge(0.95)) + \
        ggtitle('Vertical'),
    ggplot(df, aes(color='supp')) + \
        geom_errorbar(aes(y='dose', xmin='len_min', xmax='len_max'), 
                      position=position_dodgev(0.95)) + \
        ggtitle('Horizontal')
])

### Other Examples of Horizontal Orientation

In [16]:
p3 = ggplot(df, aes(y='dose', x='length', xmin='len_min', xmax='len_max', color='supp')) + \
    xlab('Tooth length [mm]')

gggrid([
    p3 + geom_crossbar(position=position_dodgev(0.95)) + ggtitle('geom_crossbar()'),
    p3 + geom_pointrange(position=position_dodgev(0.95)) + ggtitle('geom_pointrange()'),
    p3 + geom_linerange(position=position_dodgev(0.95)) + ggtitle('geom_linerange()'),
    p3 + geom_ribbon() + ggtitle('geom_ribbon()')
], ncol=2, vspace=20.0)